<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Vectorización


In [1]:
import numpy as np

In [2]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

### Datos

In [3]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

Documento 1 --> que dia es hoy \
Documento 2 --> martes el dia de hoy es martes \
Documento 3 --> martes muchas gracias

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [8]:
vector=[]
i=0
for documento in corpus:
    for termino in documento.split():
        vector.append(termino)
    print(documento.split())

vector=set(vector)
print(f"Terminos no repetidos en el corpus: {vector}")



['que', 'dia', 'es', 'hoy']
['martes', 'el', 'dia', 'de', 'hoy', 'es', 'martes']
['martes', 'muchas', 'gracias']
Terminos no repetidos en el corpus: {'muchas', 'que', 'martes', 'hoy', 'dia', 'de', 'gracias', 'el', 'es'}


### 2- OneHot encoding
Data una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [9]:
def oneHotEnconder (vector,documento):
    listOneHot=[]
    #recorre todos los terminos del vector
    for termino in vector:
        #busca el termino en el documento o texto
        if termino in documento:
            listOneHot.append(1)
        else:
            listOneHot.append(0)
    return listOneHot

In [10]:
listas=[]
for i in corpus:
    listas.append(oneHotEnconder(vector,i))
    

In [11]:
listas

[[0, 1, 0, 1, 1, 0, 0, 0, 1],
 [0, 0, 1, 1, 1, 1, 0, 1, 1],
 [1, 0, 1, 0, 0, 0, 1, 0, 1]]

### 3- Vectores de frecuencia
Data una lista de textos, devolver una matriz con la representación de frecuencia de estos

### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno